In [ ]:
from utils import load_beers, split_data
from funk_svd import SVD
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import BayesianRidge
from sklearn.neural_network import MLPRegressor
from knn import MultiCriteriaKnnModel

In [ ]:
source = load_beers("beer_reviews.csv")
train, val, test = split_data(source)
train = train.sort_values(by="user_id")
val = val.sort_values(by="user_id")
test = test.sort_values(by="user_id")


print(train["user_id"].unique().shape, train["beer_id"].unique().shape)
print(val["user_id"].unique().shape, val["beer_id"].unique().shape)
print(test["user_id"].unique().shape, test["beer_id"].unique().shape)

In [ ]:
train_feat_mats = []
for criterion in ("review_overall", "review_aroma", "review_appearance", "review_palate", "review_taste"):
    train_feat_mat = train.pivot_table(
        index="user_id",
        columns="beer_id",
        values=criterion
    ).fillna(0).values
    train_feat_mats.append(train_feat_mat)
    
val_feat_mats = []
for criterion in ("review_overall", "review_aroma", "review_appearance", "review_palate", "review_taste"):
    val_feat_mat = val.pivot_table(
        index="user_id",
        columns="beer_id",
        values=criterion
    ).fillna(0).values
    val_feat_mats.append(val_feat_mat)
    
test_feat_mats = []
for criterion in ("review_overall", "review_aroma", "review_appearance", "review_palate", "review_taste"):
    test_feat_mat = test.pivot_table(
        index="user_id",
        columns="beer_id",
        values=criterion
    ).fillna(0).values
    test_feat_mats.append(test_feat_mat)

### Single Rating Funk SVD (overall)

In [ ]:
model = SVD(learning_rate=0.001, regularization=0.005, n_epochs=100, n_factors=15, min_rating=1, max_rating=5)
new_col_names = {"user_id": "u_id", "beer_id": "i_id", "review_overall": "rating"}
model.fit(train.rename(columns=new_col_names), val.rename(columns=new_col_names), early_stopping=True, shuffle=False)

pred = model.predict(test.rename(columns=new_col_names))
rmse = mean_squared_error(test["review_overall"], pred) ** 0.5
mae = mean_absolute_error(test["review_overall"], pred)
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")

### Multiple Ratings Aggregation (aroma, appearance, palate, taste)

In [ ]:
partials = []
for criterion in ("review_aroma", "review_appearance", "review_palate", "review_taste"):
    model = SVD(learning_rate=0.001, regularization=0.005, n_epochs=100, n_factors=15, min_rating=1, max_rating=5)
    new_col_names = {"user_id": "u_id", "beer_id": "i_id", criterion: "rating"}
    model.fit(train.rename(columns=new_col_names), val.rename(columns=new_col_names), early_stopping=True, shuffle=False)
    partials.append(model.predict(test.rename(columns=new_col_names)))

In [ ]:
model = BayesianRidge()
model.fit(train[["review_aroma", "review_appearance", "review_palate", "review_taste"]].values, train["review_overall"])

pred = model.predict(list(map(list, zip(*partials))))
rmse = mean_squared_error(test["review_overall"], pred) ** 0.5
mae = mean_absolute_error(test["review_overall"], pred)
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")

In [ ]:
model = MLPRegressor(hidden_layer_sizes=(50,), solver="sgd")
model.fit(train[["review_aroma", "review_appearance", "review_palate", "review_taste"]].values, train["review_overall"])

pred = model.predict(list(map(list, zip(*partials))))
rmse = mean_squared_error(test["review_overall"], pred) ** 0.5
mae = mean_absolute_error(test["review_overall"], pred)
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")

### Single Rating KNN (overall)

In [ ]:
model = MultiCriteriaKnnModel()
model.fit([train_feat_mats[0], train_feat_mats[0]])

rmse, mae = model.test(test_feat_mats[0], 200)
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")

### Multiple Ratings KNN (aroma, appearance, palate, taste)

In [ ]:
model = MultiCriteriaKnnModel()
model.fit(train_feat_mats)

for sim_type in MultiCriteriaKnnModel.ALLOWED_SIM_TYPES:
    model.sim_type = sim_type
    rmse, mae = model.test(test_feat_mats[0], 200)
    print(f"RMSE: {rmse}")
    print(f"MAE: {mae}")